In [24]:
import numpy as np
import pandas as pd
from numpy import fft
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from warnings import filterwarnings

filterwarnings('ignore')

from modules.data_fetcher import download_historical_data

# Fetching data

In [25]:
SYMBOL= 'BTC-USDT'
window = 500
starting_index = 22000

df = download_historical_data(SYMBOL,'1hour').iloc[starting_index:starting_index+window]
df["Return"] = df.Close.pct_change()
df.dropna(inplace=True)
print(df.shape)
df.head()

(499, 8)


,Timestamp,Open,Close,High,Low,Amount,Volume,Return
Date,,,,,,,,
2020-07-05 19:00:00,1.593968e+09,9032.6,9031.3,9043.9,9027.8,45.349038,4.097564e+05,-0.000144
2020-07-05 20:00:00,1.593972e+09,9031.4,9040.7,9043.5,9030.6,35.395778,3.198915e+05,0.001041
2020-07-05 21:00:00,1.593976e+09,9040.7,9031.6,9050.5,9029.6,53.125527,4.804116e+05,-0.001007
2020-07-05 22:00:00,1.593979e+09,9031.6,9012.3,9032.4,9009.8,53.097377,4.792119e+05,-0.002137
2020-07-05 23:00:00,1.593983e+09,9012.4,9032.5,9053.9,8912.0,314.557421,2.826468e+06,0.002241


# Training and extrapolation

In [26]:
def fourrier_extrapolation(
    data_to_predict: np.array, n_predict: int, has_trend: bool = True
):
    n = data_to_predict.size
    n_harm = 50  # number of harmonics in model
    t = np.arange(0, n)
    if has_trend == True:
        p = np.polyfit(t, data_to_predict, 1)  # find linear trend in x
        x_notrend = data_to_predict - p[0] * t  # detrended x
        x_freqdom = fft.fft(x_notrend)  # detrended x in frequency domain
        f = fft.fftfreq(n)  # frequencies
        indexes = list(range(n))
        # indexes = range(n)
        # sort indexes by frequency, lower -> higher
        indexes.sort(key=lambda i: np.absolute(f[i]))

        t = np.arange(0, n + n_predict)
        restored_sig = np.zeros(t.size)
        for i in indexes[: 1 + n_harm * 2]:
            ampli = np.absolute(x_freqdom[i]) / n  # amplitude
            phase = np.angle(x_freqdom[i])  # phase
            restored_sig += ampli * np.cos(2 * np.pi * f[i] * t + phase)
        return restored_sig + p[0] * t
    else:
        x_notrend = data_to_predict  # detrended x
        x_freqdom = fft.fft(x_notrend)  # detrended x in frequency domain
        f = fft.fftfreq(n)  # frequencies
        indexes = list(range(n))
        # indexes = range(n)
        # sort indexes by frequency, lower -> higher
        indexes.sort(key=lambda i: np.absolute(f[i]))

        t = np.arange(0, n + n_predict)
        restored_sig = np.zeros(t.size)
        for i in indexes[: 1 + n_harm * 2]:
            ampli = np.absolute(x_freqdom[i]) / n  # amplitude
            phase = np.angle(x_freqdom[i])  # phase
            restored_sig += ampli * np.cos(2 * np.pi * f[i] * t + phase)
        return restored_sig

In [27]:

pred_count = 20
train_data = df.Close[:-pred_count]
test_data = df.Close[-pred_count:]

x = np.array(train_data)

n_predict = pred_count
extrapolation = fourrier_extrapolation(x, n_predict,True)

# Plot

In [28]:
fig = make_subplots(
    rows=1, cols=1, subplot_titles=("Historical price")
)

fig.add_trace(
    go.Scatter(
        name="History (train data)",
        x=np.arange(0, x.size),
        y=x,
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        name="Extrapolation (prediction)",
        x=np.arange(0, extrapolation.size),
        y=extrapolation,
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        name="History (test data)",
        x=np.arange(x.size, extrapolation.size),
        y=test_data,
    ),
    row=1,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="Fourrier extrapolation (prediction)",
)